# Import all SET metadata into MNE object

### ToC
- Import the full Matlab EEG.event struct containning `epoch_id`
- Import SET with MNE (epoched data, basic event structure)
    - Use SciPy's loadmat() to read the struct and make a pd.df ([source](https://github.com/mne-tools/mne-python/issues/3837#issuecomment-266460434) found [here](https://mne.discourse.group/t/creating-epochs-with-variable-event-latencies/3853)).
- Look into the true event df to find epochs falling into set times

### Imports

In [1]:
import mne
import os
from scipy.io import loadmat

import numpy as np
import pandas as pd

from hypyp import analyses

Define path to data directory and define subject(s)

In [2]:
path = "../../SNS_Data_Fall_2020/EEG/Cleaned_EEG/samples/"

subject_1 = "SAMPLE-1_NS_epochs.set"
subject_2 = "SAMPLE-2_NS_epochs.set"

### Making a custom event df

#### Instantiating the class

Because MNE's function `read_epochs_eeglab()` does all the work of formating the SET file's events df into a MNE-formated df, we lose important information (here, the column `epoch_id`). The class we define bellow allow to keep all these metadata and provide tools to 'cut' it to specific periodes (i.e., a time window) dynamicaly (i.e., a sliding window!). 

Bellow we import a `class` that:
- takes a SET file and read the (full) set of metadata associated to it (see `Class.df`)
- create a `Class.df_sw` which only contains events from `Class.df` which falls between `sw_min` and `sw_max`.
    - if `sw_min` and `sw_max` are defined (floats)
    - or leave `Class.df_sw` as an empty pd.Dataframe if `sw_min` and `sw_max` are `None`

> The function `read_set_events()` that enables reading MatLab's EEG.event struct embeded into a SET file into a panda df is sourced from [here](https://github.com/mne-tools/mne-python/issues/3837#issuecomment-266460434) and found [here](https://mne.discourse.group/t/creating-epochs-with-variable-event-latencies/3853).

In [3]:
from SW_utils import eeg_sw

Instantiate the class twice for 2 subject.

NB: We're not defining any sliding window parameter at the moment. 

In [4]:
# Channels to keep for analysis 
ch_to_keep_dic = [
    'Fp1', 'Fp2', 'F3', 'F4', 
    'C3', 'C4', 
    'P3', 'P4', 
    'O1', 'O2', 
    'F7', 'F8', 
    'T7', 'T8', 
    'P7', 'P8', 
    'Fz', 'Cz', 'Pz', 
    'AFz', 'CPz', 'POz']

#  We're exluding the channel 'M1' & 'M2'

In [5]:
sub1 = eeg_sw.import_set_custom(path+subject_1, sw_min=None, sw_max=None, ch_to_keep=ch_to_keep_dic)
sub2 = eeg_sw.import_set_custom(path+subject_2, sw_min=None, sw_max=None, ch_to_keep=ch_to_keep_dic)

Extracting parameters from /Users/zoubou/Documents/Scholar/Courses/UvA_M1_2/RP2_MBCS/RP2-Thesis/Analysis Thesis/MBCS_RP2_codes/SW-Approach/../../SNS_Data_Fall_2020/EEG/Cleaned_EEG/samples/SAMPLE-1_NS_epochs.set...
Not setting metadata
288 matching events found
No baseline correction applied
0 projection items activated
Ready.
Extracting parameters from /Users/zoubou/Documents/Scholar/Courses/UvA_M1_2/RP2_MBCS/RP2-Thesis/Analysis Thesis/MBCS_RP2_codes/SW-Approach/../../SNS_Data_Fall_2020/EEG/Cleaned_EEG/samples/SAMPLE-2_NS_epochs.set...
Not setting metadata
288 matching events found
No baseline correction applied
0 projection items activated
Ready.


/Users/zoubou/Documents/Scholar/Courses/UvA_M1_2/RP2_MBCS/RP2-Thesis/Analysis Thesis/MBCS_RP2_codes/SW-Approach/SW_utils/eeg_sw.py:49: RuntimeWarning: Data file name in EEG.data (hyper_cleaned_SNS_013L_014S_N_NS_sub_1 epochs.fdt) is incorrect, the file name must have changed on disk, using the correct file name (SAMPLE-1_NS_epochs.fdt).
  self.eeg = mne.io.read_epochs_eeglab(self.path).pick_channels(self.ch_to_keep, ordered=False)
/Users/zoubou/Documents/Scholar/Courses/UvA_M1_2/RP2_MBCS/RP2-Thesis/Analysis Thesis/MBCS_RP2_codes/SW-Approach/SW_utils/eeg_sw.py:49: RuntimeWarning: Data file name in EEG.data (hyper_cleaned_SNS_013L_014S_N_NS_sub_2 epochs.fdt) is incorrect, the file name must have changed on disk, using the correct file name (SAMPLE-2_NS_epochs.fdt).
  self.eeg = mne.io.read_epochs_eeglab(self.path).pick_channels(self.ch_to_keep, ordered=False)


#### Compute the Welch Power Spectral Density

In [6]:
psd1 = analyses.pow(
    sub1.eeg, 
    fmin=7.5, fmax=11, 
    n_fft=1000, 
    n_per_seg=1000, 
    epochs_average=True)
    
psd2 = analyses.pow(
    sub2.eeg, 
    fmin=7.5, fmax=11, 
    n_fft=1000, 
    n_per_seg=1000, 
    epochs_average=True)
    
data_psd = np.array([psd1.psd, psd2.psd])

Effective window size : 2.000 (s)
Effective window size : 2.000 (s)


In [7]:
psd1.psd.shape

(22, 8)

The event dataframe can now be access using `sub1.df`. However, since we didn't any specific sliding window (sw) parameter, the "cutted" `sub1.df_sw` is left empty.

In [8]:
sub1.df

,epoch,type,duration,epoch_id,latency,urevent
0,3,chan25,1,3,1,3
1,4,chan25,1,6,501,6
2,5,chan25,1,7,1001,7
3,6,chan25,1,8,1501,8
4,7,chan25,1,9,2001,9
...,...,...,...,...,...,...
283,289,chan25,1,293,141501,293
284,290,chan25,1,294,142001,294
285,291,chan25,1,295,142501,295
286,292,chan25,1,296,143001,296


In [9]:
sub1.df_sw

pandas.core.frame.DataFrame

#### Select specific events

To select a specific  range of event (e.g., from t=2 to t=16) we're using the `event_slider` method to update the class variable `sub1.df_sw`.

In [10]:
sub1.event_slider(sw_min=6, sw_max=10)
sub1.df_sw

,epoch,type,duration,epoch_id,latency,urevent
1,4,chan25,1,6,501,6
2,5,chan25,1,7,1001,7
3,6,chan25,1,8,1501,8
4,7,chan25,1,9,2001,9
5,8,chan25,1,10,2501,10


If we re-instanciate the `event_slider` method, the `sub1.df_sw` will be updated with the new boundaries.

NB: the method supports floats (although there is no need for them here)

In [11]:
sub1.event_slider(sw_min=24.4, sw_max=36.9)
sub1.df_sw

,epoch,type,duration,epoch_id,latency,urevent
17,23,chan25,1,25,8501,25
18,24,chan25,1,26,9001,26
19,25,chan25,1,27,9501,27
20,26,chan25,1,28,10001,28
21,27,chan25,1,29,10501,29
22,28,chan25,1,30,11001,30
23,29,chan25,1,31,11501,31
24,30,chan25,1,32,12001,32
25,31,chan25,1,33,12501,33
26,32,chan25,1,34,13001,34


#### Convert `df_sw` to MNE convention (`df_sw_mne`)

Here we use the `convert_to_MNE_event` to 
- grab `df_sw`
- select the column of choice
- create the (necessary) `dontuse` column, and convert it to numpy (MNE doesn't take pandas df)
- create the MNE object at stored in `sub1.mne_epo`

In [ ]:
sub1.convert_EEG_to_MNE()


> NB: the right column is the epoch column, you'll notice that these aren't the same values as the ones we selected with the `event_slider` method. That's because some have been rejected and that causes a shift.

In [15]:
sub1.eeg.events

array([[     1,      0,      1],
       [   501,      0,      1],
       [  1001,      0,      1],
       [  1501,      0,      1],
       [  2001,      0,      1],
       [  2501,      0,      1],
       [  3001,      0,      1],
       [  3501,      0,      1],
       [  4001,      0,      1],
       [  4501,      0,      1],
       [  5001,      0,      1],
       [  5501,      0,      1],
       [  6001,      0,      1],
       [  6501,      0,      1],
       [  7001,      0,      1],
       [  7501,      0,      1],
       [  8001,      0,      1],
       [  8501,      0,      1],
       [  9001,      0,      1],
       [  9501,      0,      1],
       [ 10001,      0,      1],
       [ 10501,      0,      1],
       [ 11001,      0,      1],
       [ 11501,      0,      1],
       [ 12001,      0,      1],
       [ 12501,      0,      1],
       [ 13001,      0,      1],
       [ 13501,      0,      1],
       [ 14001,      0,      1],
       [ 14501,      0,      1],
       [ 1

Now ready to be analyse for inter-brain synchrony within the time boudaries.